In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [3]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
    color: black !important;
}
sns.set(style="whitegrid")
sns.set_context(rc={"lines.linewidth": 2})

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## 不同窗口不同延迟时间下的对比

In [5]:
import dateutil
from datetime import datetime
def get_drop_latency(wm, latency, threshold):
    """
        params: wm 水印和事件的映射关系 哪一个事件时间产生了水印
                latency: 窗口被哪个水印触发
        return dict {
            "drop": 真实被丢弃的概率(一般不用),
            "late": 迟到率,
            "avg_wait": 每个窗口的平均等待时间
            "pre_win_wait": 用来具体记录每个窗口的等待时间
        }
    """
    wm = pd.read_csv(wm)
    latency = pd.read_csv(latency)
    wm.columns = ["watermark", "ts"]
    
    # drop1是真正丢的数据
    drop1 = wm[-1:].iloc[0]['watermark']/wm[-1:].iloc[0]['ts']
    # drop2是默认迟到的数据
    drop2 = wm[-2:-1].iloc[0]['watermark']/wm[-2:-1].iloc[0]['ts']
#     print(wm[-2:-1], drop2)
    latency.columns = ["watermark", "window_end"]
    latency = latency.set_index("watermark")
    wm = wm.set_index("watermark")
    res = latency.join(wm, how='inner')
    res = res.reset_index(drop = True)
    
    all_wait = 0
    pre_win_wait = []
    for index, row in res.iterrows():
        if row['ts'] - row['window_end'] > threshold+6000:
            row['ts']  = row['window_end'] + threshold
        pre_win_wait.append(row['ts'] - row['window_end'])
    
    df = pd.DataFrame(data=dict(time=res['window_end'].tolist(),wait=pre_win_wait))
    df.index = df['time'].apply(lambda x: datetime.fromtimestamp(x//1000)).tolist()
    return df

## 窗口为60s时

In [6]:
Paths = [
            "./periodic/20171004P0W60/",
            "./periodic/20171004P1W60/",
            "./periodic/20171004P2W60/",
            "./periodic/20171004P3W60/",
            "./periodic/20171004P4W60/",
           ]
WaterPaths = [path+"water.csv" for path in Paths]
TimePaths = [path+"timelatency.csv" for path in Paths]

In [7]:
adwater = './adwater/20171004AW60/water.csv'
adwait = './adwater/20171004AW60/timelatency.csv'

In [27]:
a = get_drop_latency(adwater,adwait,200000)

In [28]:
b = get_drop_latency(WaterPaths[2], TimePaths[2], 200000)

In [29]:
a['com'] = b['wait']

In [30]:
a = a.fillna(1000)

In [32]:
a.to_csv('DIDI201710.csv')